In [2]:
import geopandas as gp
import fiona
import matplotlib
import pandas as pd

In [59]:
#File from https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout#
#Run only if needed
#CENSUS_SECTIONS_INE.json should be downloaded from the above website because it's too big to host on git
confirmation = input("No need to run if already loaded. Are you sure you want to run? (y/n): ")
if confirmation.lower() == "y":
    df = gp.read_file("CENSUS_SECTIONS_INE.json")
    print("polygons loaded")
else:
    print("skipped")

No need to run if already loaded. Are you sure you want to run? (y/n): n
skipped


In [118]:
#Finding Strings for Target Municipalities
df['NMUN'][df['NMUN'].str.contains('bao')].drop_duplicates().sort_values()

33927    Bilbao
Name: NMUN, dtype: object

In [119]:
#NMUN = Málaga, Valencia, Alicante/Alacant, Madrid, Sevilla, Zaragoza, Valladolid, Bilbao, (Coruña, A)
municipality = "Bilbao"
df2 = df.query('NMUN == @municipality')

In [120]:
df2['CUDIS']

33927    4802001
33928    4802001
33929    4802001
33930    4802001
33931    4802001
          ...   
34203    4802008
34204    4802008
34205    4802008
34206    4802008
34207    4802008
Name: CUDIS, Length: 281, dtype: object

In [121]:
df2

,OBJECTID,CUSEC,CUMUN,CSEC,CDIS,CMUN,CPRO,CCA,CUDIS,OBS,...,CNUT2,CNUT3,CLAU2,NPRO,NCA,NMUN,Shape_Leng,Shape_area,Shape_len,geometry
33927,33928,4802001002,48020,002,01,020,48,16,4802001,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,1182.802274,81414.353636,1182.802274,"POLYGON ((-2.96396 43.28318, -2.96451 43.28327..."
33928,33929,4802001003,48020,003,01,020,48,16,4802001,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,888.468797,24079.631938,888.468797,"POLYGON ((-2.96166 43.28090, -2.96201 43.28155..."
33929,33930,4802001005,48020,005,01,020,48,16,4802001,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,1058.995710,49818.830643,1058.995710,"POLYGON ((-2.96029 43.28249, -2.96035 43.28260..."
33930,33931,4802001006,48020,006,01,020,48,16,4802001,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,780.143500,15550.132838,780.143500,"POLYGON ((-2.95968 43.28268, -2.95976 43.28278..."
33931,33932,4802001007,48020,007,01,020,48,16,4802001,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,1070.532013,64954.948553,1070.532013,"POLYGON ((-2.96293 43.27960, -2.96393 43.28140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,34204,4802008021,48020,021,08,020,48,16,4802008,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,959.317180,28641.850671,959.317180,"POLYGON ((-2.95120 43.25872, -2.95121 43.25863..."
34204,34205,4802008022,48020,022,08,020,48,16,4802008,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,688.763539,31938.560399,688.763539,"POLYGON ((-2.94608 43.25781, -2.94785 43.25793..."
34205,34206,4802008023,48020,023,08,020,48,16,4802008,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,597.473858,17628.915266,597.473858,"POLYGON ((-2.95006 43.25728, -2.95024 43.25776..."
34206,34207,4802008024,48020,024,08,020,48,16,4802008,,...,1,3,48020,Bizkaia,País Vasco,Bilbao,1293.583774,54746.295351,1293.583774,"POLYGON ((-2.94859 43.25665, -2.94938 43.25699..."


In [122]:
#Prepare CUDIS list to iterate through while merging polygons
unique_cudis = df2['CUDIS'].drop_duplicates()
unique_cudis

33927    4802001
33969    4802002
34003    4802003
34025    4802004
34058    4802005
34099    4802006
34145    4802007
34185    4802008
Name: CUDIS, dtype: object

In [123]:
#Define dataframe to store merged polygons
df_cudis_polygons = pd.DataFrame()

In [124]:
#Merges census section polygons to distrito level polygons
for icudis in unique_cudis:
    polygons_to_merge = df[df.CUDIS==icudis]
    merged_polygon = polygons_to_merge["geometry"].unary_union
    temp_df = pd.DataFrame({"CUDIS":[icudis],"geometry_combined":[merged_polygon]})
    print(icudis)
    df_cudis_polygons = pd.concat([df_cudis_polygons, temp_df], ignore_index=True)


4802001
4802002
4802003
4802004
4802005
4802006
4802007
4802008


In [125]:
#converts dataframe to geojson format
gdf_distrito_polygons = gp.GeoDataFrame(df_cudis_polygons,crs='epsg:4326',geometry="geometry_combined")


In [126]:
#output GEOJSON to host on github and then on metabase
confirmation = input("Make sure the file name is in line with the geo selected. Are you sure you aren't overwriting a good file (y/n): ")
if confirmation.lower() == "y":
    gdf_distrito_polygons.to_file('bilbao_distrito_polygons.geojson',driver = 'GeoJSON')
    print("file created successfully")
else:
    print("Cell execution cancelled.")

Make sure the file name is in line with the geo selected. Are you sure you aren't overwriting a good file (y/n): y
file created successfully
